**Лабораторна робота 2.2**

**студента групи ФІТ 4-4**

**Сакуна Владислава Олександровича**

Тема: створення RNN для семантичного аналізу тексту з викристанням моделі GRU

**ЗАВДАННЯ 1**

Створити RNN для семантичного аналізу тексту з використанням моделі GRU.

Розробити власну архітектуру, кількість рекурентних шарів, функції активації, вибрати оптимізатор. Отримати для двох моделей по дві рецензії (позитивну і негативну).

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, GRU, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Позитивні та негативні тексти
texts_true = [
     "Ця книга вражає своєю глибиною та проникливістю.",
    "Автор зумів майстерно розкрити головних героїв, надаючи їм життєвий досвід.",
    "Читання цієї книги залишає після себе незабутні враження та відчуття вдячності."
]

texts_false = [
    "Ця книга розчарувала своїми плоскими персонажами та передбачуваним сюжетом.",
    "Стиль автора в цій книзі відчувається дуже механічним та бездушним.",
    "Читання цієї книги не принесло жодного задоволення, лише втрату часу та розчарування."
]

texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)

# Побудова моделі
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length=max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))
history = model.fit(data_pad, np.array([[1, 0]]*count_true + [[0, 1]]*count_false), batch_size=32, epochs=50)

t = "Це неймовірно добра книга".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
res = model.predict(data_pad)

print(res)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           128000    
                                                                 
 gru (GRU)                   (None, 10, 128)           99072     
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 264450 (1.01 MB)
Trainable params: 264450 (1.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 6s 6s/step - loss: 0.6941 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 36ms

In [2]:
if res[0][0] > res[0][1]:
    print("Результат є позитивним")
else:
    print("Результат є негативним")

Результат є позитивним


In [3]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return " ".join(words)

t1 = "Книга чудова".lower()
data1 = tokenizer.texts_to_sequences([t1])
data1_pad = pad_sequences(data1, maxlen=max_text_len)
print(sequence_to_text(data1[0]))

res1 = model.predict(data1_pad)

книга
1/1 [==============================] - 0s 32ms/step


In [4]:
if res[0][0] > res[0][1]:
    print("Результат є позитивним")
else:
    print("Результат є негативним")

Результат є позитивним
